In [ ]:
!git clone https://github.com/marco-c/crashcorrelations

In [ ]:
import hashlib

from crashcorrelations import download_data, utils, crash_deviations

In [ ]:
from datetime import datetime
print(datetime.utcnow())

In [ ]:
channels = ['release', 'beta', 'aurora', 'nightly']
channel_to_versions = {}

for channel in channels:
    channel_to_versions[channel] = download_data.get_versions(channel, product='FennecAndroid')

In [ ]:
# download_data.set_token('INSERT_YOUR_TOKEN_HERE')

signatures = {}

for channel in channels:
    download_data.download_crashes(versions=channel_to_versions[channel], days=5, product='FennecAndroid')
    signatures[channel] = download_data.get_top(100, versions=channel_to_versions[channel], days=5, product='FennecAndroid')

In [ ]:
utils.mkdir('output_tmp')

totals = {
  'date': str(utils.utc_today()),
}

for channel in channels:
    print(channel)

    utils.mkdir('output_tmp/' + channel)

    df_a = crash_deviations.get_crashes(sc, versions=channel_to_versions[channel], days=5, product='FennecAndroid')

    results, total_reference, total_groups = crash_deviations.find_deviations(sc, df_a, signatures=signatures[channel])

    totals[channel] = total_reference

    for signature in signatures[channel]:
        if signature not in results:
            continue

        utils.write_json('output_tmp/' + channel + '/' + hashlib.sha1(signature).hexdigest() + '.json.gz', {
            'total': total_groups[signature],
            'results': results[signature],
        })

utils.write_json('output_tmp/all.json.gz', totals)
utils.copytree('output_tmp', 'output')

In [ ]:
from datetime import datetime
print(datetime.utcnow())